In [16]:
import pandas as pd
import os
import acquire_nick as get
import prepare_nick as prep

In [9]:
df = get.acquire_data()

Using cached CSV


In [4]:
df.dtypes

Unnamed: 0                int64
cast                     object
crew                     object
id                        int64
title                    object
genres                   object
budget                    int64
overview                 object
popularity              float64
production_companies     object
production_countries     object
revenue                 float64
runtime                 float64
vote_average            float64
vote_count              float64
keywords                 object
release_date             object
dtype: object

In [10]:
# Deleted columns Unnamed: 0
df = df.drop(['Unnamed: 0'], axis=1)


In [14]:
df.head(1)

,cast,crew,id,title,genres,budget,overview,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count,keywords,release_date
0,TomHanks TimAllen DonRickles JimVarney WallaceShawn JohnRatzenberger AnniePotts JohnMorris ErikvonDetten LaurieMetcalf R.LeeErmey SarahFreeman PennJillette,JohnLasseter JossWhedon AndrewStanton JoelCohen AlecSokolow BonnieArnold EdCatmull RalphGuggenheim SteveJobs LeeUnkrich RalphEggleston RobertGordon MaryHelenLeasman KimBlanchette MarilynMcCoppen RandyNewman DaleE.Grahn RobinCooper JohnLasseter PeteDocter JoeRanft PatsyBouge NormDeCarlo AshBrannon RandyNewman RomanFigun DonDavis JamesFlamberg MaryBethSmith RickMackay SusanBradley WilliamReeves RandyNewman AndrewStanton PeteDocter GaryRydstrom KarenRobertJackson ChrisMontan RichQuade MichaelBerenstein ColinBrady DaveyCrockettFeiten AngieGlocka RexGrignon TomK.Gurney JimmyHayward HalT.Hickel KarenKiser AnthonyB.LaMolinara GuionneLeroy BudLuckey LesMajor GlennMcQueen MarkOftedal JeffPidgeon JeffPratt SteveRabatich RogerRose SteveSegal DougSheppeck AlanSperling DougSweetland DavidTart KenWillard ThomasPorter MarkThomasHenne OrenJacob DarwynPeachey MitchPrater BrianM.Rosen SharonCalahan GalynSusman WilliamCone ShelleyDanielsLekven BobPauley BudLuckey AndrewStanton WilliamCone SteveJohnson DanHaskett TomHolloway JeanGillmore DesiréeMourad KellyO'Connell SonokoKonishi AnnM.Rockwell JulieM.McDonald RobinLee TomFreeman AdaCochavi DanaMulligan DeirdreMorrison LoriLombardo EllenDevine LaurenBethStrogoff GaryRydstrom GarySummers TimHolland PatJackson TomMyers J.R.Grubbs SusanSanford SusanPopovic DanEngstrom RuthLambert MickieMcGowan,862,Toy Story,Animation Comedy Family,30000000,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,PixarAnimationStudios,UnitedStatesofAmerica,373554033.0,81.0,7.7,5415.0,jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life,1995-10-30


Plans:
Extract the number of names inside of crew and create a column that contains a count of names that appear. Consider the same for cast.     

Scaling will be needed for the differences in scope among votes, popularity, budget, revenue, et cetera.

I'm curious as to whether or not number of keywords, or the types of keywords themselves will have a signifcant influence on the predictability of scores or budget. Presumably the film studio will have that effect, and the production company may also. 

ID should probably be cast as a category, or at the very least a generic string, since there is no rank or meaning of the order per id. 

Release date could be converted to a pandas date-time format for the sake of time-series analysis. Otherwise, it seems much of what we will be doing going forward can be framed under many different methodologies. 